In [1]:
# =====================================================
# Script Name: charts_google_search_console.ipynb
# Description: Script to create the cumulative average table from Google Search Console Data in 1st Quarter of 2024
# Author: MA
# Created Date: 2024-05-21
# Last Modified By: MA
# Last Modified Date: 2024-05-21
# Version: 1.0
# =====================================================

In [2]:
# =====================================================
# TODO: Tambahkan daftar tautan yang termasuk ke dalam 'Home'
# FIXME: ...
# =====================================================

In [3]:
# Change Log:
# 2024-05-21: Menambahkan langkah-langkah pengerjaan (MA)
# 2024-05-22: Memisahkan transformasi data CTR dari klik dan impresi (MA)

<h1>Cumulative Average of Clicks, Impressions and CTR</h1>
<p>Script ini digunakan untuk membuat salah satu Averages & Trends chart pada dashboard editorial. Selengkapnya, bisa dilihat di <a href='https://lookerstudio.google.com/reporting/ce3f7a85-abdb-46d0-b139-49e3b5c1b350/page/p_cs8tqwvbhd'><b>link berikut ini</b></a>.</p>

<h2>Tujuan dan Kegunaan</h2>
<p>Pemodelan ini dilakukan sebagai cara untuk mengevaluasi metrik utama yang ada di dashboard, yaitu `S. Google Clicks` (Sampled Google Clicks). Evaluasi ini dilakukan dengan cara membandingkan `S. Google Clicks` dengan metrik rata-ratanya dalam periode yang setara.</p>

<p> Saya menggunakan rata-rata kumulatif sebagai pembandingnya karena ia berubah sesuai dengan perubahan nilai sepanjang waktu.</p>

<h2>Siapkan Libraries dan Sumber Data</h2>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
search_df = pd.read_csv('tcid.google.technology - search_keywords.csv')
search_srt = search_df.sort_values(['Url Clicks','Date'], ascending=(False, True)).reset_index(drop=True)
search_srt

,Query,Date,Landing Page,Impressions,URL CTR,Url Clicks,Average Position
0,holi,2024-04-25,https://theconversation.com/arti-serbuk-warna-...,279229,0.002528,706,5.853343
1,kalkulator sisa umur,2024-04-11,https://theconversation.com/kalkulator-kami-da...,18486,0.032295,597,3.137509
2,holi,2024-04-26,https://theconversation.com/arti-serbuk-warna-...,233798,0.002254,527,6.025034
3,holi,2024-04-27,https://theconversation.com/arti-serbuk-warna-...,145996,0.002048,299,6.224753
4,holi,2024-04-28,https://theconversation.com/arti-serbuk-warna-...,71825,0.004038,290,5.574828
...,...,...,...,...,...,...,...
196068,the,2024-03-09,https://theconversation.com/id,500,0.000000,0,11.572000
196069,aurora,2024-03-09,https://theconversation.com/apa-penyebab-fenom...,481,0.000000,0,10.336798
196070,windy,2024-03-09,https://theconversation.com/curious-kids-what-...,1497,0.000000,0,6.474950
196071,shah rukh khan,2024-03-09,https://theconversation.com/from-deewana-to-th...,376,0.000000,0,1.000000


<h1>Langkah-langkah Pengerjaan</h1>
<p></p>Script ini ingin mendapatkan sebuah tabel yang berisi kolom-kolom sebagai berikut: `starting_date`(dt=date) , jumlah nilai (dt=int), jumlah kumulatif (dt=int), dan rata-rata kumulatif (dt=float, dibulatkan). Untuk itu, langkah-langkah pengerjaannya adalah sebagai berikut:</p>

<ul>
    <li>Bersihkan dan siapkan dataframe</li>
    <ul>
        <li>Rename kolom menjadi `date`, `landing_page`, `query`, `impression`, `ctr`, `click`, dan `avg_position` </li>
        <li>Buat variabel konfigurasi untuk melakukan subset dataframe. </li>
        <ul>
            <li>`landing_page` NOT IN (semua yang termasuk home seperti "https://theconversation.com/global", "https://theconversation.com/id", "https://theconversation.com/")</li>
            <li>`click` = 0, karena kita ingin menyetarakan analisis impresi dan klik</li>
        </ul>
        <li>Eksekusi subsetting dengan variabel konfigurasi.</li>
        <li>Tambahkan kolom `starting_date` yang berisi tanggal yang korespon dengan hari Senin di mana `date` berada (biasanya pakai DATE_DIFF() di SQL BQ).</li>
        <li>Tambahkan kolom `week_num` menggunakan isocalendar().week ke dataframe</li>
        <li>Pisahkan dan reset_index dua variabel berisi dua dataframe untuk analisis lebih jauh: klik_df (`date`, `week_num`, `landing_page`, `query`, `click`) dan impresi_df (`date`, `week_num`, `landing_page`, `query`, `impression`)</li>
        <li>Validasi: kedua dataframe memiliki jumlah rows yang sama</li>
    </ul>
    <li>Groupby masing-masing dataframe dan buat cumulative average</li>
    <ul>
        <li>Gunakan metode .groupby berdasarkan `week_num` untuk mengaggregasikan `click` dan `impression` dengan sum</li>
        <li>Simpan masing-masing transformed dataframe ke variabel-variabel khusus </li>
        <li>Buat kolom baru `cum_avg` .expanding().mean().round(1) masing-masing pada dataframe impresi dan klik</li>
        <li>Validasi: baris pertama berdasarkan `week_num` (ascending) memiliki nilai yang sama</li>
        <li>Buat dataframe CTR sebagai hasil .merge() antara dataframe kumulatif klik dan kumulatif impresi</li>
        <li>Buat kolom baru `weekly_ctr` untuk menghitung CTR per minggu</li>
        <li>Buat kolom baru `cum_avg_ctr` .expanding().mean().round(3) pada masing-masing metrik di dua variabel tadi</li>
        <li>Validasi: baris pertama berdasarkan `week_num` (ascending) memiliki nilai yang sama</li>
        <li>Validasi: tidak ada duplicate values di dataframek</li>
    </ul>
    <li>Simpan masing-masing dataframe: pd.to_csv('...')</li>
</ul>

<h2>Data Cleaning & Prep</h2>

In [6]:
# buat copy, buat variabel jumlah rows untuk validasi
df = search_srt.copy()
jumlah = search_srt.shape[0]

In [7]:
# lakukan renaming, validasi integritas data
df = search_srt.copy()
df = df.rename(columns={'Query':'query', 'Date':'date', 'Landing Page':'landing_page', 'Impressions':'impression', 'URL CTR':'ctr',\
                        'Url Clicks':'clicks', 'Average Position':'avg_position'})
print(f'Jumlah baris di df adalah {df.shape[0]}, sedangkan jumlah baris di data original adalah {jumlah}')
print(f'Kondisi dan validitas data: {df.shape[0] == jumlah}')

Jumlah baris di df adalah 196073, sedangkan jumlah baris di data original adalah 196073
Kondisi dan validitas data: True


In [8]:
#subsetting
config = {
    "landing_page_filter": ["https://theconversation.com/global", "https://theconversation.com/id", "https://theconversation.com/"],
    "url_clicks_filter": 0,
}

In [9]:
df_filtered = df[~df["landing_page"].isin(config["landing_page_filter"]) & (df["clicks"] != config["url_clicks_filter"])]
df_filtered

,query,date,landing_page,impression,ctr,clicks,avg_position
0,holi,2024-04-25,https://theconversation.com/arti-serbuk-warna-...,279229,0.002528,706,5.853343
1,kalkulator sisa umur,2024-04-11,https://theconversation.com/kalkulator-kami-da...,18486,0.032295,597,3.137509
2,holi,2024-04-26,https://theconversation.com/arti-serbuk-warna-...,233798,0.002254,527,6.025034
3,holi,2024-04-27,https://theconversation.com/arti-serbuk-warna-...,145996,0.002048,299,6.224753
4,holi,2024-04-28,https://theconversation.com/arti-serbuk-warna-...,71825,0.004038,290,5.574828
...,...,...,...,...,...,...,...
185985,penunda haid obat,2024-04-30,https://theconversation.com/tablet-penunda-hai...,7,0.142857,1,6.285714
185986,japan increase interest rate,2024-04-30,https://theconversation.com/japan-the-yen-plun...,1,1.000000,1,1.000000
185987,meteor shower southern hemisphere,2024-04-30,https://theconversation.com/meteors-supermoons...,1,1.000000,1,1.000000
185988,main judi online,2024-04-30,https://theconversation.com/mengapa-orang-bisa...,4,0.250000,1,3.750000


In [10]:
def get_monday(date):
    date = pd.to_datetime(date)
    start_of_week = date - pd.DateOffset(days=date.weekday())
    return start_of_week

def add_starting_date(df, date_column):
    df['starting_date'] = df[date_column].apply(get_monday)
    return df

In [11]:
df_filtered = add_starting_date(df_filtered, 'date')
df_filtered['week_num'] = pd.to_datetime(df_filtered['date']).dt.isocalendar().week

/var/folders/9c/r4s1k7_x1xz1jsfqzg5hg1xw0000gn/T/ipykernel_2112/3821384383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['starting_date'] = df[date_column].apply(get_monday)
/var/folders/9c/r4s1k7_x1xz1jsfqzg5hg1xw0000gn/T/ipykernel_2112/3971228059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['week_num'] = pd.to_datetime(df_filtered['date']).dt.isocalendar().week


In [12]:
df_filtered

,query,date,landing_page,impression,ctr,clicks,avg_position,starting_date,week_num
0,holi,2024-04-25,https://theconversation.com/arti-serbuk-warna-...,279229,0.002528,706,5.853343,2024-04-22,17
1,kalkulator sisa umur,2024-04-11,https://theconversation.com/kalkulator-kami-da...,18486,0.032295,597,3.137509,2024-04-08,15
2,holi,2024-04-26,https://theconversation.com/arti-serbuk-warna-...,233798,0.002254,527,6.025034,2024-04-22,17
3,holi,2024-04-27,https://theconversation.com/arti-serbuk-warna-...,145996,0.002048,299,6.224753,2024-04-22,17
4,holi,2024-04-28,https://theconversation.com/arti-serbuk-warna-...,71825,0.004038,290,5.574828,2024-04-22,17
...,...,...,...,...,...,...,...,...,...
185985,penunda haid obat,2024-04-30,https://theconversation.com/tablet-penunda-hai...,7,0.142857,1,6.285714,2024-04-29,18
185986,japan increase interest rate,2024-04-30,https://theconversation.com/japan-the-yen-plun...,1,1.000000,1,1.000000,2024-04-29,18
185987,meteor shower southern hemisphere,2024-04-30,https://theconversation.com/meteors-supermoons...,1,1.000000,1,1.000000,2024-04-29,18
185988,main judi online,2024-04-30,https://theconversation.com/mengapa-orang-bisa...,4,0.250000,1,3.750000,2024-04-29,18


In [13]:
# validasi jumlah minggu, semestinya ada 18
print(f"Nilai terendah dari week_num adalah {df_filtered['week_num'].min()}")
print(f"Nilai tertinggi dari week_num adalah {df_filtered['week_num'].max()}")

Nilai terendah dari week_num adalah 1
Nilai tertinggi dari week_num adalah 18


<h2>Grouping and Aggregating Data on Week Date</h2>

In [14]:
df_filtered

,query,date,landing_page,impression,ctr,clicks,avg_position,starting_date,week_num
0,holi,2024-04-25,https://theconversation.com/arti-serbuk-warna-...,279229,0.002528,706,5.853343,2024-04-22,17
1,kalkulator sisa umur,2024-04-11,https://theconversation.com/kalkulator-kami-da...,18486,0.032295,597,3.137509,2024-04-08,15
2,holi,2024-04-26,https://theconversation.com/arti-serbuk-warna-...,233798,0.002254,527,6.025034,2024-04-22,17
3,holi,2024-04-27,https://theconversation.com/arti-serbuk-warna-...,145996,0.002048,299,6.224753,2024-04-22,17
4,holi,2024-04-28,https://theconversation.com/arti-serbuk-warna-...,71825,0.004038,290,5.574828,2024-04-22,17
...,...,...,...,...,...,...,...,...,...
185985,penunda haid obat,2024-04-30,https://theconversation.com/tablet-penunda-hai...,7,0.142857,1,6.285714,2024-04-29,18
185986,japan increase interest rate,2024-04-30,https://theconversation.com/japan-the-yen-plun...,1,1.000000,1,1.000000,2024-04-29,18
185987,meteor shower southern hemisphere,2024-04-30,https://theconversation.com/meteors-supermoons...,1,1.000000,1,1.000000,2024-04-29,18
185988,main judi online,2024-04-30,https://theconversation.com/mengapa-orang-bisa...,4,0.250000,1,3.750000,2024-04-29,18


In [15]:
# membuat dataframe untuk klik dan impresi
click_df = df_filtered[["date", "week_num", "landing_page", "query", "clicks", "starting_date"]]
impression_df = df_filtered[["date", "week_num", "landing_page", "query", "impression", "starting_date"]]

# validasi bahwa jumlah rows di tiap dataframe adalah sama
is_valid = len(click_df) == len(impression_df)
print(f'Apakah valid? {is_valid}')

Apakah valid? True


In [16]:
click_agg = click_df.groupby("starting_date").agg(total_clicks = ('clicks', 'sum')).reset_index()
click_agg["cum_avg"] = click_agg["total_clicks"].expanding().mean().round(1)
impression_agg = impression_df.groupby("starting_date").agg(total_impressions = ('impression', 'sum')).reset_index()
impression_agg["cum_avg"] = impression_agg["total_impressions"].expanding().mean().round(1)

In [17]:
ctr_agg = pd.merge(click_agg[['starting_date','total_clicks']], impression_agg[['starting_date','total_impressions']], on='starting_date',\
                   how='inner')
ctr_agg['weekly_ctr'] = (ctr_agg['total_clicks'] / ctr_agg['total_impressions']).round(4)

In [18]:
ctr_agg['cum_avg_ctr'] = ctr_agg['weekly_ctr'].expanding().mean().round(4)
ctr_agg

,starting_date,total_clicks,total_impressions,weekly_ctr,cum_avg_ctr
0,2024-01-01,14177,388662,0.0365,0.0365
1,2024-01-08,18010,381107,0.0473,0.0419
2,2024-01-15,17556,354844,0.0495,0.0444
3,2024-01-22,16535,314969,0.0525,0.0464
4,2024-01-29,16646,307394,0.0542,0.0480
5,2024-02-05,16156,279259,0.0579,0.0496
6,2024-02-12,19959,598318,0.0334,0.0473
7,2024-02-19,15580,314309,0.0496,0.0476
8,2024-02-26,14569,263480,0.0553,0.0485
9,2024-03-04,13474,260177,0.0518,0.0488


In [21]:
# Validasi: baris pertama berdasarkan `week_num` (ascending) memiliki nilai yang sama
is_valid = click_agg['starting_date'].min()==impression_agg['starting_date'].min()==ctr_agg['starting_date'].min()
print(f'Kelengkapan dan keutuhan dataframe yang dimiliki adalah {is_valid}')

Kelengkapan dan keutuhan dataframe yang dimiliki adalah True


<h2>Saving the Processed Dataframe into .csv Files</h2>

In [20]:
click_agg.to_csv('cumavg_klik.csv', index=False)
impression_agg.to_csv('cumavg_imp.csv', index=False)
ctr_agg.to_csv('cumavg_ctr.csv', index=False)